In [ ]:
# 구글 마운트 시키기
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# 필요한 모듈 불러오기
import numpy as np
import pandas as pd
import matplotlib
import requests
import sys
import json
import datetime
from matplotlib import font_manager,rc  #한글 폰트 입력을 위한 라이브러리
import matplotlib.pyplot as plt   #그래프 그리기 위한 라이브러리
plt.rc('font', family = 'NanumBarunGothic')


In [ ]:
# 데이터 불러오기
df_all = pd.read_csv('/content/gdrive/MyDrive/2023-1 DScover_스터디_프로젝트/Data/서울시 중구 불법주정차 위반 단속 CCTV 위치정보.csv', encoding = 'cp949')
df_all

,고정형CCTV지번주소,위도,경도,자치구,단속지점명,현장구분
0,봉래1가 126-2,37.558311,126.971875,중구,오일뱅크,불법주정차구역
1,서울 중구 신당동 52-52,37.563513,127.018296,중구,다산어린이공원 앞,불법주정차구역
2,충무로2가 64-5,37.561095,126.986610,중구,국민은행 명동역지점,불법주정차구역
3,남산동3가 13-31,37.560804,126.986398,중구,남산초교,불법주정차구역
4,저동2가 88-13,37.564572,126.988855,중구,백병원 건너편,불법주정차구역
...,...,...,...,...,...,...
145,서울 중구 퇴계로87길 13,37.565713,127.021144,중구,미담어린이집,불법주정차구역
146,서울 중구 명동22-13,37.561295,126.983144,중구,중앙우체국 뒤 사거리,불법주정차구역
147,서울 중구 을지로 281,37.566248,127.010901,중구,DDP디자인갤러리P5입구,불법주정차구역
148,서울 중구 필동2가 116-1,37.557660,126.995650,중구,필동어린이집,불법주정차구역


In [ ]:
def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': 'KakaoAK {}'.format(APP_KEY)}
    resp = requests.get(url, headers=headers)
    # print('%s : success for request [%s]' % (datetime.now(), url))
    return resp.text


def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[0]
        json_name = json_doc.get('address_name')
        json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name, json_code

 
def get_address(x,y):
    address = []        
    json_name, json_code = reverse_geocode(x, y) 
    address.append(json_name)
    return address # 전처리 함수에서 주소 리스트 받아서 데이터프레임에 추가


def get_code(x,y):
    code = []        
    json_name, json_code = reverse_geocode(x, y) 
    code.append(json_code)
    return code # 전처리 함수에서 행정구역코드 리스트 받아서 데이터프레임에 추가

In [ ]:
APP_KEY = 'ec0383ca1cd6fd424a7c0416595e6e87' 
URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'

df = df_all[['단속지점명', '위도', '경도']]

for i in range(len(df)): 
    x_crd = float(df.loc[i, ['경도']])
    y_crd = float(df.loc[i, ['위도']])
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    df.loc[i, ['ADDRESS']] = address
    df.loc[i, ['CODE']] = code
    
    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

<ipython-input-6-202e911ac3f9>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, ['ADDRESS']] = address
<ipython-input-6-202e911ac3f9>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, ['CODE']] = code


0 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [ ]:
df

,단속지점명,위도,경도,ADDRESS,CODE
0,오일뱅크,37.558311,126.971875,서울특별시 중구 봉래동1가,1114011900
1,다산어린이공원 앞,37.563513,127.018296,서울특별시 중구 신당동,1114016200
2,국민은행 명동역지점,37.561095,126.986610,서울특별시 중구 충무로2가,1114012500
3,남산초교,37.560804,126.986398,서울특별시 중구 남산동3가,1114013000
4,백병원 건너편,37.564572,126.988855,서울특별시 중구 저동2가,1114016100
...,...,...,...,...,...
145,미담어린이집,37.565713,127.021144,서울특별시 중구 황학동,1114016500
146,중앙우체국 뒤 사거리,37.561295,126.983144,서울특별시 중구 충무로1가,1114012400
147,DDP디자인갤러리P5입구,37.566248,127.010901,서울특별시 중구 을지로7가,1114014900
148,필동어린이집,37.557660,126.995650,서울특별시 중구 필동2가,1114013800


In [ ]:
df['CODE'].unique()

array(['1114011900', '1114016200', '1114012500', '1114013000',
       '1114016100', '1114011000', '1114015500', '1114016800',
       '1114015900', '1114013400', '1114015000', '1114015200',
       '1114017100', '1114013500', '1114015400', '1114014700',
       '1114013900', '1114014500', '1114014900', '1114016400',
       '1114011100', '1114016600', '1114016500', '1114016000',
       '1114015100', '1114015300', '1114012800', '1114014800',
       '1114012200', '1114010900', '1114012100', '1114014400',
       '1114010500', '1114015800', '1114012600', '1114017200',
       '1114016300', '1114016900', '1114010400', '1114013100',
       '1114012900', '1114011400', '1114013800', '1114011300',
       '1114013200', '1114013300', '1114013600', '1114016700',
       '1114017400', '1114017300', '1114010700', '1114011800',
       '1114014600', '1114017000', '1114012400'], dtype=object)

In [ ]:
data_list = df["ADDRESS"].str.split(' ')
data_list

0      [서울특별시, 중구, 봉래동1가]
1        [서울특별시, 중구, 신당동]
2      [서울특별시, 중구, 충무로2가]
3      [서울특별시, 중구, 남산동3가]
4       [서울특별시, 중구, 저동2가]
              ...        
145      [서울특별시, 중구, 황학동]
146    [서울특별시, 중구, 충무로1가]
147    [서울특별시, 중구, 을지로7가]
148     [서울특별시, 중구, 필동2가]
149      [서울특별시, 중구, 신당동]
Name: ADDRESS, Length: 150, dtype: object

In [ ]:
df['법정동'] = data_list.str.get(2)

In [ ]:
df['CODE'].nunique()

55

In [ ]:
df

,단속지점명,위도,경도,ADDRESS,CODE,법정동
0,오일뱅크,37.558311,126.971875,서울특별시 중구 봉래동1가,1114011900,봉래동1가
1,다산어린이공원 앞,37.563513,127.018296,서울특별시 중구 신당동,1114016200,신당동
2,국민은행 명동역지점,37.561095,126.986610,서울특별시 중구 충무로2가,1114012500,충무로2가
3,남산초교,37.560804,126.986398,서울특별시 중구 남산동3가,1114013000,남산동3가
4,백병원 건너편,37.564572,126.988855,서울특별시 중구 저동2가,1114016100,저동2가
...,...,...,...,...,...,...
145,미담어린이집,37.565713,127.021144,서울특별시 중구 황학동,1114016500,황학동
146,중앙우체국 뒤 사거리,37.561295,126.983144,서울특별시 중구 충무로1가,1114012400,충무로1가
147,DDP디자인갤러리P5입구,37.566248,127.010901,서울특별시 중구 을지로7가,1114014900,을지로7가
148,필동어린이집,37.557660,126.995650,서울특별시 중구 필동2가,1114013800,필동2가


In [ ]:
df.to_csv("/content/gdrive/MyDrive/2023-1 DScover_스터디_프로젝트/Data/junggu_illege.csv", encoding = "cp949")

In [ ]:
df["법정동"].unique()

array(['봉래동1가', '신당동', '충무로2가', '남산동3가', '저동2가', '수표동', '을지로3가', '순화동',
       '초동', '인현동2가', '을지로4가', '주교동', '중림동', '예관동', '오장동', '쌍림동', '필동3가',
       '광희동1가', '을지로7가', '무학동', '소공동', '서소문동', '황학동', '인현동1가', '을지로5가',
       '방산동', '남산동1가', '을지로6가', '회현동2가', '장교동', '회현동1가', '장충동2가', '을지로2가',
       '충무로3가', '명동1가', '의주로2가', '흥인동', '의주로1가', '을지로1가', '저동1가', '남산동2가',
       '태평로2가', '필동2가', '북창동', '충무로4가', '충무로5가', '묵정동', '정동', '만리동2가',
       '만리동1가', '삼각동', '남대문로5가', '광희동2가', '충정로1가', '충무로1가'], dtype=object)

In [ ]:
# code의 데이터 타입을 object에서 int64로 바꾸기
print(df.dtypes)
print()
df1 = df.astype({'CODE' : 'int64'})
print(df1.dtypes)

단속지점명       object
위도         float64
경도         float64
ADDRESS     object
CODE        object
법정동         object
dtype: object

단속지점명       object
위도         float64
경도         float64
ADDRESS     object
CODE         int64
법정동         object
dtype: object


In [ ]:
# 중구 행정동, 법정도 코드 
junggu = pd.read_csv('/content/gdrive/MyDrive/2023-1 DScover_스터디_프로젝트/Data/junggu_mix.csv', encoding = 'utf-8')
junggu

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1114000000,서울특별시,중구,NaN,1114000000,중구,19880423,NaN
1,1114052000,서울특별시,중구,소공동,1114011100,소공동,19880423,NaN
2,1114052000,서울특별시,중구,소공동,1114011300,북창동,19880423,NaN
3,1114052000,서울특별시,중구,소공동,1114011400,태평로2가,19880423,NaN
4,1114052000,서울특별시,중구,소공동,1114011500,남대문로2가,19880423,NaN
...,...,...,...,...,...,...,...,...
88,1114067000,서울특별시,중구,황학동,1114016500,황학동,19880423,NaN
89,1114068000,서울특별시,중구,중림동,1114017100,중림동,19880423,NaN
90,1114068000,서울특별시,중구,중림동,1114017200,의주로2가,19880423,NaN
91,1114068000,서울특별시,중구,중림동,1114017300,만리동1가,19880423,NaN


In [ ]:
junggu = junggu.iloc[: , [0, 3, 4, 5]]
junggu

,행정동코드,읍면동명,법정동코드,동리명
0,1114000000,NaN,1114000000,중구
1,1114052000,소공동,1114011100,소공동
2,1114052000,소공동,1114011300,북창동
3,1114052000,소공동,1114011400,태평로2가
4,1114052000,소공동,1114011500,남대문로2가
...,...,...,...,...
88,1114067000,황학동,1114016500,황학동
89,1114068000,중림동,1114017100,중림동
90,1114068000,중림동,1114017200,의주로2가
91,1114068000,중림동,1114017300,만리동1가


In [ ]:
junggu.columns = ['행정동 코드', '행정동명', 'CODE', '법정동']
junggu.columns

Index(['행정동 코드', '행정동명', 'CODE', '법정동'], dtype='object')

In [ ]:
junggu.dtypes

행정동 코드     int64
행정동명      object
CODE       int64
법정동       object
dtype: object

In [ ]:
junggu.nunique()

행정동 코드    16
행정동명      15
CODE       1
법정동       75
dtype: int64

'봉래동1가', '신당동', '충무로2가', '남산동3가', '저동2가', '수표동', '을지로3가', '순화동',
       '초동', '인현동2가', '을지로4가', '주교동', '중림동', '예관동', '오장동', '쌍림동', '필동3가',
       '광희동1가', '을지로7가', '무학동', '소공동', '서소문동', '황학동', '인현동1가', '을지로5가',
       '방산동', '남산동1가', '을지로6가', '회현동2가', '장교동', '회현동1가', '장충동2가', '을지로2가',
       '충무로3가', '명동1가', '의주로2가', '흥인동', '의주로1가', '을지로1가', '저동1가', '남산동2가',
       '태평로2가', '필동2가', '북창동', '충무로4가', '충무로5가', '묵정동', '정동', '만리동2가',
       '만리동1가', '삼각동', '남대문로5가', '광희동2가', '충정로1가', '충무로1가']

In [ ]:
junggu[junggu["법정동"] == "봉래동1가"]

,행정동 코드,행정동명,CODE,법정동
7,1114052000,소공동,1114011900,봉래동1가
17,1114054000,회현동,1114011900,봉래동1가


In [ ]:
junggu['법정동'].unique()

array(['중구', '소공동', '북창동', '태평로2가', '남대문로2가', '남대문로3가', '남대문로4가', '봉래동1가',
       '서소문동', '정동', '순화동', '의주로1가', '충정로1가', '남창동', '남대문로5가', '봉래동2가',
       '회현동1가', '회현동2가', '충무로1가', '무교동', '다동', '태평로1가', '을지로1가', '을지로2가',
       '남대문로1가', '삼각동', '수하동', '장교동', '수표동', '회현동3가', '충무로2가', '명동1가',
       '명동2가', '남산동1가', '남산동2가', '남산동3가', '저동1가', '예장동', '충무로4가', '충무로5가',
       '묵정동', '필동1가', '필동2가', '필동3가', '남학동', '주자동', '장충동2가', '충무로3가',
       '장충동1가', '인현동2가', '예관동', '광희동1가', '광희동2가', '쌍림동', '을지로6가', '을지로7가',
       '오장동', '을지로4가', '을지로5가', '주교동', '방산동', '을지로3가', '입정동', '산림동', '초동',
       '인현동1가', '저동2가', '신당동', '흥인동', '무학동', '황학동', '중림동', '의주로2가',
       '만리동1가', '만리동2가'], dtype=object)

In [ ]:
df2_merge = pd.merge(left = df1, right = junggu, how= "inner", on = "법정동")
df2_merge

,단속지점명,위도,경도,ADDRESS,CODE_x,법정동,행정동 코드,행정동명,CODE_y
0,오일뱅크,37.558311,126.971875,서울특별시 중구 봉래동1가,1114011900,봉래동1가,1114052000,소공동,1114011900
1,오일뱅크,37.558311,126.971875,서울특별시 중구 봉래동1가,1114011900,봉래동1가,1114054000,회현동,1114011900
2,다산어린이공원 앞,37.563513,127.018296,서울특별시 중구 신당동,1114016200,신당동,1114061500,신당동,1114011900
3,다산어린이공원 앞,37.563513,127.018296,서울특별시 중구 신당동,1114016200,신당동,1114062500,다산동,1114011900
4,다산어린이공원 앞,37.563513,127.018296,서울특별시 중구 신당동,1114016200,신당동,1114063500,약수동,1114011900
...,...,...,...,...,...,...,...,...,...
340,플러스무역앞,37.564375,127.009387,서울특별시 중구 광희동2가,1114014600,광희동2가,1114059000,광희동,1114011900
341,제일스포츠앞,37.564554,127.008748,서울특별시 중구 광희동2가,1114014600,광희동2가,1114059000,광희동,1114011900
342,농협중앙회 앞,37.565650,126.968167,서울특별시 중구 충정로1가,1114017000,충정로1가,1114052000,소공동,1114011900
343,중앙우체국 뒤 사거리,37.561295,126.983144,서울특별시 중구 충무로1가,1114012400,충무로1가,1114054000,회현동,1114011900


In [ ]:
df2_merge.to_csv("/content/gdrive/MyDrive/2023-1 DScover_스터디_프로젝트/Data/junggu_duplicated.csv", encoding = "cp949")

In [ ]:
df2_merge[df2_merge.duplicated(subset = ["단속지점명"])]

,단속지점명,위도,경도,ADDRESS,CODE_x,법정동,행정동 코드,행정동명,CODE_y
1,오일뱅크,37.558311,126.971875,서울특별시 중구 봉래동1가,1114011900,봉래동1가,1114054000,회현동,1114011900
3,다산어린이공원 앞,37.563513,127.018296,서울특별시 중구 신당동,1114016200,신당동,1114062500,다산동,1114011900
4,다산어린이공원 앞,37.563513,127.018296,서울특별시 중구 신당동,1114016200,신당동,1114063500,약수동,1114011900
5,다산어린이공원 앞,37.563513,127.018296,서울특별시 중구 신당동,1114016200,신당동,1114064500,청구동,1114011900
6,다산어린이공원 앞,37.563513,127.018296,서울특별시 중구 신당동,1114016200,신당동,1114065000,신당제5동,1114011900
...,...,...,...,...,...,...,...,...,...
329,코리아빌딩 입구앞 사거리,37.563023,126.998482,서울특별시 중구 충무로4가,1114013200,충무로4가,1114059000,광희동,1114011900
331,미니스톱 앞,37.561228,126.998058,서울특별시 중구 충무로5가,1114013300,충무로5가,1114059000,광희동,1114011900
333,제일병원 주차타워 앞,37.562285,126.999196,서울특별시 중구 묵정동,1114013600,묵정동,1114058000,장충동,1114011900
337,중림어린이집 앞,37.555812,126.967584,서울특별시 중구 만리동1가,1114017300,만리동1가,1114068000,중림동,1114011900
